In [5]:
import numpy as np

N, D_in, H, D_out = 64, 1000, 100, 10

x = np.random.randn(N, D_in)
y = np.random.randn(N, D_out)

w1 = np.random.randn(D_in, H)
w2 = np.random.randn(H, D_out)

learning_rate = 1e-6
for t in range(500):

  h = x.dot(w1)
  h_relu = np.maximum(h,0)
  y_pred = h_relu.dot(w2)

  loss = np.square(y_pred - y).sum()
  if t%20 == 0 :
    print(t, loss)

  grad_y_pred = 2.0 * (y_pred - y)
  grad_w2 = h_relu.T.dot(grad_y_pred)
  grad_h_relu = grad_y_pred.dot(w2.T)
  grad_h = grad_h_relu.copy()
  grad_h[h < 0] = 0
  grad_w1 = x.T.dot(grad_h)

  w1 -= learning_rate*grad_w1
  w2 -= learning_rate*grad_w2

0 28681977.68861396
20 201051.52962570245
40 24530.447714442533
60 4843.878378056897
80 1216.9217172426588
100 353.9219164391362
120 113.04810087185648
140 38.264993856304
160 13.44598927859824
180 4.844902630420139
200 1.7765039676626455
220 0.6597042603690644
240 0.24730106051860462
260 0.09337615558246881
280 0.0354560213609587
300 0.013523531858849521
320 0.005176573018296956
340 0.0019873155538535467
360 0.0007647753224652684
380 0.0002948949533178057
400 0.0001138954351602402
420 4.405050443786634e-05
440 1.7056576124227512e-05
460 6.6109556211657675e-06
480 2.564514363099933e-06


---

-> numpy를 이용한 fully_connected nn 구현</br>
-> forward, backward 직접구현

---



In [7]:
import torch

dtype = torch.float
device = torch.device("cpu")
device = torch.device("cuda:0")

N, D_in, H, D_out = 64, 1000, 100, 10

x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device, dtype=dtype)

learning_rate = 1e-6
for t in range(500):

  h = x.mm(w1)
  h_relu = h.clamp(min=0)
  y_pred = h_relu.mm(w2)

  loss = (y_pred - y).pow(2).sum().item()
  if t%100 == 99 :
    print(t, loss)


  grad_y_pred = 2.0 * (y_pred - y)
  grad_w2 = h_relu.t().mm(grad_y_pred)
  grad_h_relu = grad_y_pred.mm(w2.t())
  grad_h = grad_h_relu.clone()
  grad_h[h<0] = 0
  grad_w1 = x.t().mm(grad_h)

  w1 -= learning_rate * grad_w1
  w2 -= learning_rate * grad_w2

99 517.314453125
199 2.6916236877441406
299 0.030974093824625015
399 0.0007234406075440347
499 9.509991650702432e-05



---
-> pytorch를 이용한 fully_connected nn 구현</br>
-> forward, backward 직접구현

---



In [8]:
import torch

class MyReLU(torch.autograd.Function):

  @staticmethod
  def forward(ctx, input) : 

    ctx.save_for_backward(input)
    return input.clamp(min=0)

  @staticmethod
  def backward(ctx, grad_output):

    input, = ctx.saved_tensors
    grad_input = grad_output.clone()
    grad_input[input < 0] = 0
    return grad_input

dtype = torch.float
device = torch.device("cpu")
device = torch.device("cuda:0")

N, D_in, H, D_out = 64, 1000, 100, 10

x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(500):

  relu = MyReLU.apply

  y_pred = relu(x.mm(w1)).mm(w2)

  loss = (y_pred - y).pow(2).sum()
  if t % 100 == 99 :
    print(t, loss.item())


  loss.backward()

  with torch.no_grad():
    w1 -= learning_rate * w1.grad
    w2 -= learning_rate * w2.grad
    
    w1.grad.zero_()
    w2.grad.zero_()



99 744.2021484375
199 5.368178367614746
299 0.06526672095060349
399 0.0012079041916877031
499 0.00011554277443792671


99 523.8118286132812
199 3.4765267372131348
299 0.04136575385928154
399 0.0008464562124572694
499 9.080549352802336e-05
